In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../3-Data-Ingestion/speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [7]:
docs

[Document(metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='â€¦\n\nIt 

In [10]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(docs,embeddings)
db

In [13]:
### querying
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [20]:
retriever = db.as_retriever()
retriever.invoke(query)
docs[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [21]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='1bb980b7-09c6-42ac-a395-610a6e8b6699', metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(0.6290264)),
 (Document(id='ea2fb513-ba71-4fe6-8270-d0eef7276f5f', metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentle

In [22]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.029584666714072227,
 -0.022738471627235413,
 -0.016281655058264732,
 0.025944307446479797,
 -0.04286843538284302,
 -0.08759701251983643,
 -0.040611620992422104,
 0.019588658586144447,
 0.002717164345085621,
 -0.0078486492857337,
 0.016555441543459892,
 0.003051814856007695,
 -0.019494833424687386,
 0.03404109552502632,
 -0.022798212245106697,
 -0.023937121033668518,
 -0.041795749217271805,
 -0.012135960161685944,
 0.014812310226261616,
 0.002868998097255826,
 -0.001957785105332732,
 -0.02365417405962944,
 -0.035131845623254776,
 0.022166946902871132,
 -0.006545314099639654,
 0.015857554972171783,
 -0.03465509042143822,
 0.01180412620306015,
 0.0655784085392952,
 -0.006505156867206097,
 -0.009974385611712933,
 0.03142382204532623,
 -0.0023667362984269857,
 -0.03968833014369011,
 0.025159655138850212,
 -0.03655576705932617,
 0.024765020236372948,
 -0.11073465645313263,
 0.007832562550902367,
 -0.009223454631865025,
 0.014653961174190044,
 0.016739070415496826,
 0.03231857344508171,
 -

In [23]:
docs_score = db.similarity_search_by_vector(embedding_vector)

In [24]:
### Saving and Loading
db.save_local("false_index")

In [27]:
new_db =  FAISS.load_local("false_index",embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [28]:
docs

[Document(id='1bb980b7-09c6-42ac-a395-610a6e8b6699', metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ea2fb513-ba71-4fe6-8270-d0eef7276f5f', metadata={'source': '../3-Data-Ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I